# Dice-Sum Emergence Demo

This interactive demo uses our Emergence–Ridge framework to rediscover the *most likely sum* when rolling $d$ fair six-sided dice via two tree-based metrics—no probability theory required.

**1. Collapse potential**  
$$
E_{\downarrow}(d)
\;=\;
\sum_{s=d}^{6d}
\ln\bigl(P(\text{sum}=s)\cdot 6^d \;+\; 1\bigr)
$$

**2. Branching complexity**  
$$
E_{\uparrow}(d)
\;=\;
d \,\ln(7)
$$

**3. Emergence Gap** and **Discrete Curvature**  
$$
\Delta(d)
\;=\;
E_{\downarrow}(d)
\;-\;
E_{\uparrow}(d),
\quad
\Phi(d)
\;=\;
\Delta(d+1)
-2\,\Delta(d)
+\Delta(d-1).
$$

A single sharp spike in **$\Phi(d)$** pinpoints the critical number of dice where the mode “emerges” most strongly. For six-sided dice this correctly picks **$d=2$** (sum = 7).

In [1]:
import math, itertools
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def compute_phi_and_mode(max_d, sides):
    deltas = []
    modes  = []
    for d in range(1, max_d+1):
        # tally sums
        counts = {}
        for roll in itertools.product(range(1, sides+1), repeat=d):
            s = sum(roll)
            counts[s] = counts.get(s, 0) + 1

        # collapse-potential per-sum: log(count+1)
        log_terms = {s: math.log(c+1) for s, c in counts.items()}
        # mode from the largest term
        mode_sum = max(log_terms, key=log_terms.get)
        modes.append(mode_sum)

        # E_up and E_down
        E_up   = d * math.log(sides + 1)
        E_down = sum(log_terms.values())
        deltas.append(E_down - E_up)

    # discrete curvature
    phis = [0.0]
    for i in range(1, len(deltas)-1):
        phis.append(deltas[i+1] - 2*deltas[i] + deltas[i-1])
    phis.append(0.0)

    return np.array(phis), np.array(modes)

def plot_interactive_emergence(max_d=6, sides_list=(4,6,8)):
    ds = list(range(1, max_d+1))

    # First, print a table to validate numbers
    table = {'d': ds}
    for s in sides_list:
        phis, modes = compute_phi_and_mode(max_d, s)
        table[f'mode_{s}'] = modes
        table[f'phi_{s}']  = phis
    df = pd.DataFrame(table)
    print("Validation table (d, mode, phi):\n", df.to_string(index=False))

    # Now the interactive plot
    fig = go.Figure()
    for sides in sides_list:
        phis, modes = compute_phi_and_mode(max_d, sides)
        fig.add_trace(go.Scatter(
            x=ds,
            y=modes,
            mode='lines+markers',
            name=f'{sides}-sided mode',
            marker=dict(size=8),
            hovertemplate=(
                '<b>Dice:</b> '+str(sides)+'-sided<br>'+
                '<b>d:</b> %{x}<br>'+
                '<b>Mode:</b> %{y}<br>'+
                '<b>Φ:</b> %{customdata:.3f}<extra></extra>'
            ),
            customdata=phis
        ))

    fig.update_layout(
        title='Emergence as Most Probably Sum on Dice',
        xaxis_title='Number of Dice',
        yaxis_title='Most Common Sum',
        hovermode='closest'
    )
    fig.show()

if __name__ == "__main__":
    plot_interactive_emergence(max_d=8, sides_list=(4,6,8))


Validation table (d, mode, phi):
  d  mode_4    phi_4  mode_6     phi_6  mode_8     phi_8
 1       1 0.000000       1  0.000000       1  0.000000
 2       5 4.817890       7 11.704416       9 20.268836
 3       7 5.668044      10 12.996177      13 21.693583
 4      10 5.902130      14 13.107885      18 21.638098
 5      12 5.943532      17 13.098048      22 21.612244
 6      15 5.953550      21 13.113815      27 21.651096
 7      17 5.964112      24 13.143066      31 21.703602
 8      20 0.000000      28  0.000000      36  0.000000


# Rule 110 Emergence Demo

This demo applies the Emergence–Ridge framework to Wolfram’s Rule 110 cellular automaton (width $N=12$, inverse‐branch depth $d=4$).  We sweep the random seed density $\lambda\in[0,1]$ and compute:

1. **Collapse potential**  
   $$
     E_{\downarrow}(\lambda)
     = \mathbb{E}_{\text{seed}}\bigl[\sum_{\text{labels }a}
       \ln\bigl(\#\{\text{preimages of }a\}+1\bigr)\bigr]
   $$

2. **Branching complexity**  
   $$
     E_{\uparrow}(\lambda)
     = \mathbb{E}_{\text{seed}}\bigl[\sum_{\text{nodes }}
       \ln\bigl(\deg(\text{node})+1\bigr)\bigr]
   $$

3. **Harmonic norm**  
   $$
     G(\lambda)
     = \frac{2}{\displaystyle\frac{1}{E_{\downarrow}(\lambda)}
               +\frac{1}{E_{\uparrow}(\lambda)}}
   $$

A single sharp peak in **$G(\lambda)$** pinpoints the seed density at which Rule 110’s inverse‐branch structure is most “balanced” between collapse and branching. Below is the code to generate and plot this curve.  


In [ ]:
#!/usr/bin/env python3
"""
Emergence–Ridge Demo: Rule 110 Cellular Automaton
Computes the harmonic norm of (E↓, E↑) vs. seed density λ.
"""

import numpy as np
import matplotlib.pyplot as plt
import random
from mpmath import mp, mpf, log, fsum
from collections import Counter
from itertools import product

# ─── USER‐TUNABLE CONSTANTS ───
mp.dps       = 60       # precision
CA_LEN       = 12       # CA width
DEPTH_CA     = 4        # inverse‐branch depth
CA_SAMPLES   = 16       # seeds per λ
GRID         = 600      # λ samples
RULE_CODE    = 110      # Wolfram code
EPS          = 1e-8     # clamp for zero‐avoidance
# ──────────────────────────────

# Build Rule-110 table
table = {
    tuple(int(bit) for bit in f"{i:03b}"):
    int(bit)
    for i, bit in enumerate(f"{RULE_CODE:08b}"[::-1])
}

def step(bits: str) -> str:
    N = len(bits)
    return ''.join(
        str(table[(
            int(bits[(i-1) % N]),
            int(bits[i]),
            int(bits[(i+1) % N])
        )])
        for i in range(N)
    )

# Precompute preimages
preimage_map = {}
for bits in product('01', repeat=CA_LEN):
    s = ''.join(bits)
    c = step(s)
    preimage_map.setdefault(c, []).append(s)

def ca_branches(state, _=None):
    return preimage_map.get(state, [])

def ca_attr(state, _=None):
    s = state
    for _ in range(60):
        s = step(s)
    return s

def random_seed(lam):
    return ''.join('1' if random.random()<lam else '0'
                   for _ in range(CA_LEN))

def E_down(seed):
    nodes = [seed]
    for _ in range(DEPTH_CA):
        nodes = [c for y in nodes for c in ca_branches(y)]
    labels = Counter(ca_attr(n) for n in nodes)
    return float(fsum(log(count+1) for count in labels.values()))

def E_up(seed):
    total = mp.mpf('0')
    nodes = [seed]
    for _ in range(DEPTH_CA):
        nxt = []
        for y in nodes:
            br = ca_branches(y)
            total += log(len(br)+1)
            nxt.extend(br)
        nodes = nxt
    return float(total)

def harmonic(a, b):
    a = max(a, EPS)
    b = max(b, EPS)
    return 2.0/(1.0/a + 1.0/b)

# Sweep λ
lambdas = np.linspace(0.05, 0.95, GRID)
G = np.empty_like(lambdas)
for i, lam in enumerate(lambdas):
    eds, eus = [], []
    for _ in range(CA_SAMPLES):
        seed = random_seed(lam)
        eds.append(E_down(seed))
        eus.append(E_up(seed))
    ed, eu = np.mean(eds), np.mean(eus)
    G[i] = harmonic(ed, eu)

# Plot
plt.figure()
plt.plot(lambdas, G, lw=1.5)
plt.xlabel("Seed density $\lambda$")
plt.ylabel("Harmonic norm")
plt.title(f"Rule 110 (N={CA_LEN}, depth={DEPTH_CA})")
plt.grid(alpha=0.4)
plt.tight_layout()
plt.show()


## Quickstart: Using `EmergenceRidge`

The `EmergenceRidge` class automates the “ridge‐tracing” procedure on any finitely‐branching rewrite system.  In three steps you:

1. **Define a collapse weight**  
   A function `cw(term) → ℝ` measuring “collapse potential” of a term.

2. **Provide your rewrite rules**  
   A list of pairs `[(ℓ₁, r₁), (ℓ₂, r₂), …]` describing candidate bidirectional rewrites.

3. **Choose a seed term**  
   The starting term `x_star` whose inverse‐branch tree you wish to analyze.

```python
from emergence_ridge import EmergenceRidge

# 1) Collapse‐weight: e.g., shorter terms “collapse” more strongly
collapse_metric = lambda s: -len(s)

# 2) Candidate equations
rules = [("ab", "ba"), ("aba", "a"), ("aa", "")]

# 3) Base term
x0 = "ababa"

# Initialize and run
engine = EmergenceRidge(
    collapse_weight=collapse_metric,
    candidate_eqs=rules,
    x_star=x0,
    depth_limit=5,      # max inverse‐branch depth
    max_meta_depth=2,   # levels of meta‐analysis
)
result = engine.run()


In [ ]:
import time
import math
from functools import lru_cache

class EmergenceRidge:
    def __init__(
        self,
        collapse_weight,
        candidate_eqs,
        x_star,
        max_steps=50,
        depth_limit=5,
        delta=1e-2,
        eta=0.1,
        tol=1e-3,
        max_meta_depth=2,
        join_steps=2
    ):
        self.cw = collapse_weight
        self.candidate_eqs = list(candidate_eqs)
        self.x_star = x_star
        self.max_steps = max_steps
        self.depth_limit = depth_limit
        self.delta = delta
        self.eta = eta
        self.tol = tol
        self.max_meta_depth = max_meta_depth
        self.join_steps = join_steps

        self.completed_rules = []
        self.ops = []
        self.grammar = {}
        self.regime = {}

    def critical_pairs(self, l1, r1, l2, r2):
        cps = []
        minlen = min(len(l1), len(l2))
        for k in range(1, minlen+1):
            if l1.endswith(l2[:k]):
                s1 = r1 + l2[k:]
                s2 = l1[:-k] + r2
                cps.append((s1, s2))
            if l2.endswith(l1[:k]):
                s1 = r2 + l1[k:]
                s2 = l2[:-k] + r1
                cps.append((s1, s2))
        return cps

    def apply_rules(self, term, R):
        results = set()
        for l, r in R:
            idx = term.find(l)
            while idx != -1:
                new_term = term[:idx] + r + term[idx+len(l):]
                results.add(new_term)
                idx = term.find(l, idx+1)
        return results

    def joinable(self, s1, s2, R):
        reach1, reach2 = {s1}, {s2}
        for _ in range(self.join_steps):
            next1 = set()
            for t in reach1:
                next1 |= self.apply_rules(t, R)
            reach1 |= next1
            next2 = set()
            for t in reach2:
                next2 |= self.apply_rules(t, R)
            reach2 |= next2
            if reach1 & reach2:
                return True
        return bool(reach1 & reach2)

    def complete_rules(self):
        R = []
        for l, r in self.candidate_eqs:
            if self.cw(l) > self.cw(r):
                R.append((l, r))
            else:
                R.append((r, l))
        while True:
            new = []
            n = len(R)
            for i in range(n):
                for j in range(i+1, n):
                    l1, r1 = R[i]; l2, r2 = R[j]
                    for s1, s2 in self.critical_pairs(l1, r1, l2, r2):
                        if not self.joinable(s1, s2, R):
                            if self.cw(s1) > self.cw(s2):
                                new.append((s1, s2))
                            else:
                                new.append((s2, s1))
            to_add = [rule for rule in new if rule not in R]
            if not to_add:
                break
            R.extend(to_add)
        self.completed_rules = R
        self.ops = [(lambda lh, rh: (lambda x: rh if x == lh else x))(l, r) for l, r in R]

    @lru_cache(maxsize=None)
    def compute_metrics(self, state, depth):
        if depth <= 0:
            return 0.0, 0.0
        ed = eu = 0.0
        succs = [op(state) for op in self.ops if op(state) != state]
        if not succs:
            return 0.0, 0.0
        n = len(succs)
        for y in succs:
            w_down = max(self.cw(y), 0.0)
            w_up = math.log(n + 1) if n > 0 else 0.0
            ed_c, eu_c = self.compute_metrics(y, depth - 1)
            ed += w_down + ed_c
            eu += w_up + eu_c
        return ed, eu

    def compute_delta_and_phi(self, state):
        E_down, E_up = [], []
        for d in range(self.depth_limit + 1):
            ed, eu = self.compute_metrics(state, d)
            E_down.append(ed)
            E_up.append(eu)
        Δ = [E_down[i] - E_up[i] for i in range(len(E_down))]
        phi = [0.0] * len(Δ)
        for i in range(1, len(Δ) - 1):
            phi[i] = Δ[i+1] - 2 * Δ[i] + Δ[i-1]
        return Δ, phi

    def trace_ridge(self, state):
        Δ, phi = self.compute_delta_and_phi(state)
        p = float(self.depth_limit) / 2
        while True:
            idx = int(round(p))
            fwd = phi[min(len(phi) - 1, idx + 1)]
            grad = (fwd - phi[idx])
            p_new = p + self.eta * grad
            if abs(p_new - p) < self.tol:
                break
            p = max(1.0, min(self.depth_limit - 1, p_new))
        return int(round(p))

    def build_grammar(self, state=None, depth=0):
        if state is None:
            state = self.x_star
            self.complete_rules()
        if depth > self.max_meta_depth:
            return

        d_crit = self.trace_ridge(state)
        ed, eu = self.compute_metrics(state, d_crit)
        Δ = ed - eu

        regime = 'collapse' if Δ > 0 else 'branching'
        self.regime[(state, depth)] = regime

        succs = [op(state) for op in self.ops if op(state) != state]
        if regime == 'branching':
            children = sorted(set(succs))
        else:
            best_y, best_phi = None, -math.inf
            for y in succs:
                _, phi_y = self.compute_delta_and_phi(y)
                peak = max(phi_y)
                if peak > best_phi:
                    best_phi, best_y = peak, y
            children = [best_y] if best_y is not None else []

        self.grammar[(state, depth)] = children
        for child in children:
            self.build_grammar(child, depth + 1)

    def run(self):
        t0 = time.time()
        self.complete_rules()
        self.build_grammar()
        return {
            "completed_rules": self.completed_rules,
            "grammar": self.grammar,
            "regime": self.regime,
            "timings": {"total": time.time() - t0}
        }

# Execute and display results
collapse_metric = lambda s: -len(s)
eqs = [("ab", "ba"), ("aba", "a"), ("aa", "")]
x0 = "ababa"

engine = EmergenceRidge(
    collapse_weight=collapse_metric,
    candidate_eqs=eqs,
    x_star=x0,
    depth_limit=5,
    max_meta_depth=2
)
result = engine.run()

print("Completed Rules:", result["completed_rules"])
print("Grammar:", result["grammar"])
print("Regime:", result["regime"])
print("Timings:", result["timings"])